In [ ]:
# Alp Ozkayikci, Nezir Uran, Jennifer Warbeck 
# UPF CSIM 2025: Systems Design, Integration and Control 

# Simulating the Relationship between Tadpoles and Dragonfly Larvae in Freshwater Ecosystems
#### Course: Systems Design, Integration and Control (2025)
**Project by Alp Ozkayikci, Nezir Uran, Jennifer Warbeck**

### Background
A study done by Takahara et al (2004) explored how Hyla japonica tadpoles detect and respond to predator cues from the dragonfly nymph Anax parthenope julius. The tadpoles reduced their activity levels when exposed to chemical cues from the predator, but didn’t react to visual cues. Interestingly, tadpoles could detect these chemical signals from a greater distance than the dragonfly could detect the tadpoles, suggesting an evolutionary advantage for the tadpoles in avoiding predators. This difference in detection range highlights how chemical cue detection helps shape predator-prey dynamics in aquatic environments, enabling tadpoles to initiate defensive behaviors before becoming visible to their predators.

In another study by Mogali et al (2023), it was found that tadpoles of Clinotarsus curtipes assess predation risk using chemical cues. The tadpoles showed no response to kairomones from unfed Pantala flavescens larvae but exhibited strong antipredator behaviors—such as reduced swimming and increased burst speed—when exposed to water containing predator excreta. Their strongest response occurred when predators had consumed conspecific tadpoles, indicating their ability to gauge predation risk and adjust defense behaviors accordingly.

![Dragonfly Larvae preying Tadpoles](https://images.fineartamerica.com/images-medium-large-5/dragonfly-larva-preying-on-tadpoles-nigel-cattlin.jpg)
Fig. 1: Dragonfly Larvae preying on Tadpoles

# The Simulation
#### Work distribution
We worked on this project together in a few sessions after class, pair programming style. Everyone contributed to the research and the coding.

## Predator-Prey Dynamics in the Simulation

### Agent Types:
1. **Tadpoles** 🟡  
   - Have a **(almost) circular vision field** (detect predators via chemical cues).  
   - **Slow down when sensing a dragonfly** (fails in 10% of encounters).  
   - If they **stop moving, they become invisible** to the dragonfly.

2. **Dragonfly Larvae** 🔺  
   - Have a **triangular vision field** (can only see directly ahead).  
   - **Can only detect moving tadpoles** and will chase and eat them.  
   - **If it doesn't eat for 10 seconds, it dies** (gets destroyed).

### Key Dynamics:
- **Hunger Dynamic**: Dragonfly larvae must hunt tadpoles to survive.  
- **Fear Dynamic**: Tadpoles detect dragonflies and slow down to avoid being eaten.


![Simulation Model](images/idea.png)
Fig. 2: First concept sketch of the agent types (dragonfly larvae & tadpoles) with vision field

### Setup Vivarium

In [1]:
from vivarium.controllers.notebook_controller import NotebookController
from vivarium.utils.handle_server_interface import start_server_and_interface, stop_server_and_interface

## Simulation Setup
#### Before running this simulation for the first time (or if you made changes to that file):
Move `dragonfly_tadpole_scene.yaml` to your `vivarium/conf/scene` Folder. 

From the terminal, run the following command:

`cp dragonfly_tadpole_scene.yaml (PATH TO vivarium/conf/scene)`

Then, you can proceed.

#### Start the server and init the controller with the agents

In [19]:
start_server_and_interface(scene_name="dragonfly_tadpole_scene")


 Found the process scripts/run_interface.py running with this PID: 86607
 Found the process scripts/run_server.py running with this PID: 86599



Stopping server and interface processes




The following processes are running:
 - Interface (PIDs: ['86607'])
 - Server (PIDs: ['86599'])
Do you want to stop them? (y/n):  y


Killed process with PID: 86607
Killed process with PID: 86599

Server and Interface processes have been stopped



Received signal 15, shutting down
/Users/jennifer.warbeck/vivarium/vivarium/utils

STARTING SERVER
[2025-02-04 16:54:41,201][__main__][INFO] - Scene running: dragonfly_tadpole_scene
[2025-02-04 16:54:43,108][vivarium.simulator.simulator][INFO] - Simulator initialized

STARTING INTERFACE


2025-02-04 16:54:46,980 Starting Bokeh server version 3.3.4 (running on Tornado 6.4.2)
2025-02-04 16:54:46,981 User authentication hooks NOT provided (default user enabled)
2025-02-04 16:54:46,983 Bokeh app running at: http://localhost:5006/run_interface
2025-02-04 16:54:46,983 Starting Bokeh server with process id: 86746


In [20]:
controller = NotebookController()

/Users/jennifer.warbeck/miniconda3/lib/python3.12/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


In [21]:
controller.agents

In [8]:
controller.run()

Dragonfly Sensor Data: Left=0.0, Right=-0.0
No tadpoles detected. Patrolling...
Dragonfly Sensor Data: Left=0.0, Right=-0.0
No tadpoles detected. Patrolling...
Simulator is already stopped


### Tadpole behavior `tadpole_behavior(agent)`

- Simulates how a **tadpole** reacts to sensing a **dragonfly larva**.
- Uses **sensors** to detect `DRAGONFLY_LARVAE` on the left or right.
- Normally, the tadpole moves forward at **half speed** (`left_motor = 0.5`, `right_motor = 0.5`).
- If it detects a dragonfly larva (**left > 0 or right > 0**):
  - It **stops moving** (`left_motor = 0`, `right_motor = 0`) as a defense mechanism.

In [22]:
def tadpole_behavior(agent):
    """Tadpole slows down when sensing a dragonfly larvae."""
    
    left, right = agent.sensors(sensed_entities=["DRAGONFLY_LARVAE"])
    
    left_motor = 0.5
    right_motor = 0.5
    if left > 0 or right > 0:
        left_motor = 0
        right_motor = 0
    return left_motor, right_motor

### Dragonfly behavior `dragonfly_behavior(agent)`

- Simulates how a **dragonfly larva hunts tadpoles**.
- Uses **sensors** to detect `TADPOLES` on the left or right.
- Moves at a **base speed** of `0.5`.
- If a tadpole is detected (**left > 0 or right > 0**):
  - **Increases speed** by `0.2`.
  - **Turns towards the detected tadpole** by adjusting motor speeds:
    - If a tadpole is on the left → **turns left** (`right_motor > left_motor`).
    - If a tadpole is on the right → **turns right** (`left_motor > right_motor`).
- If no tadpoles are detected, it **moves straight** at the base speed.
- Motor values are **clamped between 0.0 and 1.0** to prevent exceeding limits.
- Defines **diet as "TADPOLES"**, indicating it preys on them.
- Tadpoles that are stationary (not moving) are ignored

In [23]:
def dragonfly_behavior(agent):
    """Dragonfly larvae hunt only moving tadpoles; ignores stationary ones and always moves."""

    # Get sensor readings for nearby tadpoles
    left, right = agent.sensors(sensed_entities=["TADPOLES"])
    base_speed = 0.5  # Maintain constant movement

    # Debugging: Print sensor data
    print(f"Dragonfly Sensor Data: Left={left}, Right={right}")

    # Default movement (always move forward)
    left_motor = base_speed
    right_motor = base_speed

    # Filter out stationary tadpoles from being eaten
    moving_tadpoles = []
    
    # Check if tadpoles are detected
    if left > 0 or right > 0:
        print("Dragonfly detects tadpoles!")

        # Simulate checking for movement (approximate, since we don't have direct access to tadpole motor values)
        if left > 0.05 or right > 0.05:  # If the detected tadpole is "strongly" detected, assume it's moving
            print("Chasing moving tadpoles!")
            base_speed += 0.2  # Increase speed when hunting
            left_motor = base_speed + (right - left)  # Turn towards right if sensing on left
            right_motor = base_speed + (left - right)  # Turn towards left if sensing on right
            
            # Only consider moving tadpoles for eating
            moving_tadpoles.append("TADPOLES")

        else:
            print("Detected a stationary tadpole. Ignoring...")

    else:
        print("No tadpoles detected. Patrolling...")

    # Prevent dragonflies from eating stationary tadpoles
    agent.diet = moving_tadpoles  # Only consume moving tadpoles

    # Ensure motor values stay within range [0, 1.0]
    left_motor = max(0.0, min(left_motor, 1.0))
    right_motor = max(0.0, min(right_motor, 1.0))

    return left_motor, right_motor

### **Asynchronous Hunger Monitoring: `monitor_hunger(agent)`**

The function `monitor_hunger(agent)` is an **asynchronous coroutine** that monitors whether a **dragonfly larva** has eaten within the last 50 seconds. If it hasn’t, the function removes the agent from the simulation.

#### **How It Works:**
- The function runs **recursively** in an infinite loop (`while True`).
- It **pauses execution** for **50 seconds** using `await asyncio.sleep(50)`.
- It then checks if the agent has eaten in the last **50 seconds** using `agent.has_eaten_since(50)`.
  - If **`False`** (meaning the dragonfly **has not eaten**), it is removed (`agent.exists = False`).
  - The function then **exits (`return`)**, stopping further execution.

#### **Key Features:**
- **Asynchronous Execution:** The use of `async def` and `await asyncio.sleep(50)` allows the function to run **without blocking** other tasks.
- **Hunger Monitoring:** Continuously checks if the agent has eaten and removes it if it hasn’t.
- **Automatic Removal:** Once the hunger threshold is exceeded, the dragonfly larva **is removed from the simulation**, preventing inactive agents from persisting.


In [11]:
import asyncio

async def monitor_hunger(agent):
    """Recursively checks if the dragonfly larvae has eaten in the last 10 seconds.
       If not, removes the agent from the simulation.
    """
    hunger_limit = 60.0  # Max time (seconds) without eating

    while True:
        await asyncio.sleep(50)  # Wait 10 seconds before checking condition

        
        # If hunger limit exceeded, remove the agent
        if agent.has_eaten_since(50) == False:
            agent.exists = False
            return  # Stop the coroutine after removal


## **Initializing Tadpoles and Dragonfly Larvae Behavior**


### **Step 1: Assigning Agents to Groups**
- `tadpoles = [controller.agents[0], controller.agents[1], controller.agents[2], controller.agents[3]]`  
  - Selects the **first four agents** and assigns them as **tadpoles**.
- `dragonflies = [controller.agents[4], controller.agents[5]]`  
  - Selects the **next two agents** and assigns them as **dragonfly larvae**.

In [24]:
tadpoles = [controller.agents[0], controller.agents[1], controller.agents[2], controller.agents[3]]
dragonflies = [controller.agents[4], controller.agents[5]]

### **Step 2: Configuring Tadpoles**
- The loop `for tadpole in tadpoles:` iterates over all tadpoles.
  - **Removes all previous behaviors** using `tadpole.detach_all_behaviors()`.
  - **Attaches the tadpole behavior** using `tadpole.attach_behavior(tadpole_behavior)`, enabling them to detect dragonfly larvae.
  - **Sets their movement speed** (`left_motor = 0.5`, `right_motor = 0.5`) to move forward at a normal pace.


In [13]:
for tadpole in tadpoles:
    tadpole.detach_all_behaviors()
    tadpole.attach_behavior(tadpole_behavior)
    tadpole.left_motor = 0.5
    tadpole.right_motor = 0.5

### **Step 3: Configuring Dragonflies**
- The loop `for dragonfly in dragonflies:` iterates over all dragonfly larvae.
  - **Removes all previous behaviors** using `dragonfly.detach_all_behaviors()`.
  - **Attaches the dragonfly behavior** using `dragonfly.attach_behavior(dragonfly_behavior)`, allowing them to hunt tadpoles.
  - **Starts the hunger monitoring task** asynchronously with `asyncio.create_task(monitor_hunger(dragonfly))`, ensuring that dragonfly larvae are removed from the simulation if they don’t eat within the defined hunger limit.
  - **Sets their movement speed** (`left_motor = 0.5`, `right_motor = 0.5`) to move forward at a base speed.

In [14]:
for dragonfly in dragonflies:
    dragonfly.detach_all_behaviors()
    dragonfly.attach_behavior(dragonfly_behavior)
    asyncio.create_task(monitor_hunger(dragonfly))
    dragonfly.left_motor = 0.5
    dragonfly.right_motor = 0.5

### **Step 4: Enabling the Eating Mechanism**
- `controller.start_eating_mechanism()`  
  - This function **activates the eating system**, allowing predator-prey interactions where dragonfly larvae can consume tadpoles.

In [17]:
controller.start_eating_mechanism()

#### Stop the controller

In [1]:
controller.stop()

NameError: name 'controller' is not defined

# **Discussion**
## **Main Problems we ran into**
- **Jupyter Notebook has Layout Issues sometimes when trying to add a new section** → This made the development and documentation process longer.
- **The dragonfly_tadpole_scene.yaml File** → It is a pain having to move it to the vivarium Folder each time when you make changes to that file
- **The checking of the life energy of the dragonfly** → Implementing an `async def monitor_hunger(agent)` was not intutive at first

## **Current Limitations of the Simulation**  
- **Performance issues** → Our laptops sometimes crashed due to high processing demands.  
- **Dragonfly tracking is too persistent** → Once a dragonfly sees a moving tadpole, it keeps chasing even if the tadpole stops, giving the predator an advantage.  
- **Dragonflies move too slowly** → They often die before catching prey, making survival difficult.  
- **Hunger system could be improved** → We spent a lot of time debugging the hunger mechanic and ended up using an asynchronous function, but it doesn’t feel like the cleanest solution—there might be a better approach.  


## **Future Improvements of the Simulation**  
- **More tadpoles, smaller vision field** → Helps dragonflies survive longer by increasing prey density.  
- **Tadpole escape behavior** → Instead of just slowing down, tadpoles could randomly dart away.  
- **Add obstacles / hiding spots for tadpoles**

### References
1. Mogali, S. M., Shanbhag, B. A., & Saidapur, S. K. (2023). Behavioral responses of tadpoles of Clinotarsus curtipes (Anura: Ranidae) to odor cues of dragonfly larvae. Phyllomedusa, 22(1), 11–20. https://doi.org/10.11606/issn.2316-9079.v22i1p11-20
2. Teruhiko Takahara, Yukihiro Kohmatsu, Atsushi Maruyama, Hideyuki Doi, Hiroki Yamanaka, Ryohei Yamaoka, Inducible defense behavior of an anuran tadpole: cue-detection range and cue types used against predator, Behavioral Ecology, Volume 23, Issue 4, July-August 2012, Pages 863–868, https://doi.org/10.1093/beheco/ars044